## All in One

In [1]:
import numpy as np
import pandas as pd


# train = pd.read_csv(f"../workdir/AIS-ModelFrames/aisdk-2023-11-08-xs_2_kkn_train_{known}.csv")


#filename = "aisdk-2023-11-08_1_knn"
filename = "aisdk-2023-11-09_1_knn"
# filename = 'aisdk-2023-11-08-xs_3_knn'
#filename = "aisdk-2023-11-08-s_2_knn"
#filename = "aisdk-2023-11-08-6xs_2_knn"

#df = pd.read_csv("../workdir/AIS-ModelFrames/"+filename+"_test_"+str(known)+"_0.csv")



def generate_noise(df, known, noise):
    df = df.copy()
    i = 0
    while i < (known+1):
        range = np.random.normal(0, noise, len(df.index))

        angles = np.random.uniform(0, 2 * np.pi, len(df.index))

        # range = range.reshape(len(df.index), known+1)
        # angles = angles.reshape(len(df.index), known+1)

        # print(type(range))
        # print(range.shape)

        noiseFrame = pd.DataFrame(range, columns=['range']) 
        angleFrame = pd.DataFrame(angles, columns=['angle'])
        noiseFrame = pd.concat([noiseFrame, angleFrame], axis=1)



        noiseFrame[f"noise-x-{str(i)}"] = noiseFrame['range'] * np.cos(noiseFrame['angle']) # Offset - X
        noiseFrame[f'noise-y-{str(i)}'] = noiseFrame['range'] * np.sin(noiseFrame['angle']) # Offset - Y
        noiseFrame.drop(['range', 'angle'], axis=1, inplace=True)

        if i == 0:
            noiseFrame1 = noiseFrame
        else:
            noiseFrame1 = pd.concat([noiseFrame1, noiseFrame], axis=1)

        i = i + 1 

    #print(noise)
    del noiseFrame
    noiseFrame1.head()

    i = 1
    while i < (known):
        df[f'input_{i}_x'] = df[f'input_{i}_x'] + noiseFrame1[f'noise-x-{i}'] - noiseFrame1['noise-x-0']
        df[f'input_{i}_y'] = df[f'input_{i}_y'] + noiseFrame1[f'noise-y-{i}'] - noiseFrame1['noise-y-0']
        i = i + 1
    df[f'output_x'] = df['output_x'] + noiseFrame1[f'noise-y-{i}'] - noiseFrame1['noise-x-0']
    df[f'output_y'] = df['output_y'] + noiseFrame1[f'noise-y-{i}'] - noiseFrame1['noise-y-0']

    df.head()
    return df

import numpy as np
from pathlib import Path



noise_level = 0, 5, 10
for known in range(3, 13, 1):
    for noise in noise_level:
        #path = Path(f"../workdir/AIS-ModelFrames/{filename}_test_{known}_{noise}.csv")
        path = Path(f"../workdir/AIS-ModelFrames/{filename}_test_{known}_{noise}.csv")
        if not path.exists():     
            df = pd.read_csv(f"../workdir/AIS-ModelFrames/{filename}_test_{known}_0.csv")
            test_model = generate_noise(df, known, noise)
            test_model.to_csv(path, index=False)
            print(f"Done with {known} {noise}")

Done with 3 5
Done with 3 10
Done with 4 5
Done with 4 10
Done with 5 5
Done with 5 10
Done with 6 5
Done with 6 10
Done with 7 5
Done with 7 10
Done with 8 5
Done with 8 10
Done with 9 5
Done with 9 10
Done with 10 5
Done with 10 10
Done with 11 5
Done with 11 10
Done with 12 5
Done with 12 10
